In [1]:
from Functions.polygon_scrape import *
from Functions.RF_Funcs import *
import pandas as pd
import numpy as np
from secret import polygon_key
from datetime import datetime, date
import polygon as pg
import warnings
warnings.filterwarnings('ignore')
import time

In [2]:
import pandas_market_calendars as mcal
nyse = mcal.get_calendar('NYSE')
start_date = '2020-01-01'
end_date = '2023-12-31'
schedule = nyse.schedule(start_date=start_date, end_date=end_date)
trading_days = schedule['market_open'].dt.date.tolist()

### Acquire Model Predictions

In [3]:
# data = pd.read_csv('Treasury.csv')

# X = data[['RV_(t-1)','RV_t Weekly', 'RV_t Monthly',
#           'r_(t-1)', 'r_t Weekly', 'r_t Monthly',
#           'VIX Close (t-1)', '1MT (t-1)']]
# Y = data['RV_t (Realized Variance)']

# X['1MT (t-1)'] = pd.to_numeric(X['1MT (t-1)'], errors='coerce')
# X['1MT (t-1)'] = X['1MT (t-1)'].fillna((X['1MT (t-1)'].shift(+1) + X['1MT (t-1)'].shift(-1)) / 2)

# op = RollingWindowRF(X,Y,data['date'])
# pred = np.sqrt(op['predictions'].loc['values'])*np.sqrt(365)

### Import data to avoid long runtime

In [3]:
pred = pd.read_csv('predictions.csv').set_index('date')['values']
print(np.average(pred))

0.13223407478542387


### Deploy the Option Strategy Using the Prediction

In [4]:
data = pd.read_csv('Treasury.csv')

Dates where Tu, Th expiry options are NOT available

In [5]:
dates = data['date'].iloc[-250:-220]

res = []

for date in dates:
    try:
        res.append(OptionStrategy(model_estimate=pred[date],date=datetime.strptime(date,'%m/%d/%y').date(), trading_days=trading_days))
    except LookupError:
        res.append('Data Not Found')
    print(f'{date}: {res[-1]}')
    time.sleep(62)

8/12/22: {'Profit': 294.948385210142, 'Investment': 468.8252801028862}
8/15/22: Data Not Found
8/16/22: {'Profit': -1.4654418935466822, 'Investment': 39.58134774120297}
8/17/22: Data Not Found
8/18/22: {'Profit': -146.55649714544023, 'Investment': 126.97819285202009}
8/19/22: {'Profit': -5987.310371427113, 'Investment': 4417.9620128798415}
8/22/22: Data Not Found
8/23/22: {'Profit': 56.963196812984066, 'Investment': 73.66470549818449}
8/24/22: Data Not Found
8/25/22: {'Profit': -141.5032515100578, 'Investment': 68.61383005377459}
8/26/22: {'Profit': 2.8319054629442673, 'Investment': 6.000089696647396}
8/29/22: Data Not Found
8/30/22: {'Profit': 23.77891601424836, 'Investment': 65.44585600543628}
8/31/22: Data Not Found
9/1/22: {'Profit': 2.38834218696978, 'Investment': 13.75351910853782}
9/2/22: {'Profit': 2214.5923323434095, 'Investment': 2883.2196816696955}
9/6/22: {'Profit': -334.4073319069166, 'Investment': 643.046937829198}
9/7/22: Data Not Found
9/8/22: {'Profit': -124.8825478034

Dates where M-F expiry options ARE available

In [6]:
dates = data['date'].iloc[-180:-150]

res = []

for date in dates:
    try:
        res.append(OptionStrategy(model_estimate=pred[date],date=datetime.strptime(date,'%m/%d/%y').date(), trading_days=trading_days))
    except LookupError:
        res.append('Data Not Found')
    print(f'{date}: {res[-1]}')
    time.sleep(62)

11/21/22: {'Profit': -1.8923188672908586, 'Investment': 2.074198743249799}
11/22/22: {'Profit': 15.386626531654798, 'Investment': 57.81962196720342}
11/23/22: {'Profit': -15.44514989992795, 'Investment': 15.44514989992795}
11/25/22: {'Profit': -291.2249067346175, 'Investment': 191.49707250387343}
11/28/22: {'Profit': 275.6632872902714, 'Investment': 275.6632872902714}
11/29/22: {'Profit': -928.4366349045295, 'Investment': 442.82771463561227}
11/30/22: {'Profit': 56.80312106208814, 'Investment': 56.80312106208814}
12/1/22: {'Profit': 6.539167206289468, 'Investment': 6.6946315503567035}
12/2/22: {'Profit': -227.324123708408, 'Investment': 98.68181102044107}
12/5/22: {'Profit': -1197.0124410651836, 'Investment': 1115.4586108342103}
12/6/22: {'Profit': 44.7954453011827, 'Investment': 44.7954453011827}
12/7/22: {'Profit': 13.041652388287119, 'Investment': 121.27248162927853}
12/8/22: {'Profit': 1.5267224126292849, 'Investment': 4.44284525156762}
12/9/22: {'Profit': -1218.7563561413447, 'Inv